In [37]:
import STARTHER
import nvdbapiv3
import pandas as pd
import geopandas as gpd
from shapely import wkt
import xmltodict
from xml.parsers.expat import ExpatError
import json
import requests
import re
import numpy as np

# Lærdommer så langt - 134119  systemobjekter og 118740 vegobjekter

  * Vi har ett og kun ett systemobjekt per mor-objekt 
  * Hvert systemobjekt har kun ei mor _(tre stk unntak, som vi ignorerer (datafeil))_
  * Vi har ca 800 systemobjekter som er mor-løse. _Mest sannsynlig er dette objekter der mor er sletta uten kaskadesletting, alternativt så kan det være registreringstabber eller tekniske feil_. 
  * "Beskrivelse" - inneholder som regel kun ett enkelt punkt, aldri ei linje, ei heller to punkt (fra- og til). 
     - Det normale er at punktet er registrert som   "frapos"
     - 1167  ganger er dette punktet registrert som  "tilpos". 
     - Men aldri både frapos og tilpos i samme element.
  * **Beskrivelse-feltet er trunkert for 9% (12332) av objektene** (XML-dokument avsluttes midt inne i en tagg). 
    * Et flertall av disse lar seg lese med regulære uttrykk
    * 1241 av de trunkerte beskrivelsene har to øst,nord, høyde oppføringer. Vi ser ingen logikk i hvilket punkt som er rett, eller om evt det kan gi mening i å bruke begge punktene (f.eks som innløp og utløp på ei stikkrenne). _dette må evt granskes nærmere_. 
    
  * De aller, aller fleste vegobjektene ligger samme sted som systemobjektene, men vi har noen (veldig) få der vegobjektets plassering på vegnettet avviker fra systemobjektets plassering. Avstanden i luftlinje er fra 0 - 300m (men dette gjelder veldig få tilfeller)

# Veien videre 

  * De analyserte systemobjektene kvalitetskontrolleres og filtreres
    * Fjerne alt som er tvilsomt, objekter uten mor-relasjon etc. 
    * Kvalitetsjekk på geometri: Ikke for stor avstand mellom objektets stedfesting og geometrien. 
      * **FØRSTE UTKAST** Kun objekter der vi har funnet punkt-geometri der egengeometri < 10m fra senterlinjegeometri og der differansen mellom avstand oppgitt i metadata og avstand mellom geometriene ikke er større enn 5 m. (Dvs hvis det står 3m avstand i metadata og vi finner 10m avstand så tas ikke dette objektet med). => 119265 objekter / 89% av alle systemobjektene. 
  * Senere kan vi  titte litt nærmere på de 1241 objektene som muligens kan ha linje-geometri
    * Tvilsomt, men verdt en titt
    * Lengden må jo være ganske kort for at det skal være meningsfyllt å ha ei linje med kun 2 koordinatpunkt. 
  * De kvalitetskontrollerte systemobjektene kobles med sitt mor-objekt etter gitte regler
    * Vi ignorerer mor-objekter som allerede har en eller annen form for egengeometri
      * Det tilfører f.eks ingen verdi å gi punktgeometri til ei stikkrenne som allerede har linjegeometri eller flategeometri
    * Vi må sjekke at det ikke er for stor avstand mellom mor og datter.  
    
Kun de objektene som passerer alle kvalitetskontrollene blir med videre for å lage endringssett der mor-objektene som ikke har noen form for geometri blir oppdatert med punkt-geometri hentet fra systemobjektet.  

Det ser ut som om de 1241 objektene med to nord+øst oppføringer må forkastes. Skal granske dette litt mer, men tviler på om det er informasjon her av verdi, evt hvordan vi kan skille ut verdiløs informasjon fra det vi ønsker å lagre til NVDB. 

# Analyse av systemobjekt

In [2]:
sok = nvdbapiv3.nvdbFagdata( 794)
mydf = pd.DataFrame( sok.to_records( relasjoner=True ))

Eksport av 134147 objekter kommer til å ta tid...
Objekt 1000 av 134147
Objekt 5000 av 134147
Objekt 10000 av 134147
Objekt 20000 av 134147
Objekt 30000 av 134147
Objekt 40000 av 134147
Objekt 50000 av 134147
Objekt 60000 av 134147
Objekt 70000 av 134147
Objekt 80000 av 134147
Objekt 90000 av 134147
Objekt 100000 av 134147
Objekt 110000 av 134147
Objekt 120000 av 134147
Objekt 130000 av 134147
Manglende geometri-element for 7 vegobjekter fra dette søket
nvdbFagdata: Søkeobjekt for vegobjekter fra NVDB api V3
ObjektType: 794 Systemobjekt
Filtere
{}
Parametre som styrer responsen:
{
    "inkluder": [
        "alle"
    ]
}
Statistikk fra NVDB api V3
{
    "antall": 134147,
    "lengde": 0.0
}
Pagineringsinfo: Antall objekt i databuffer= 0
{
    "antall": 1000,
    "hvilken": 1,
    "meredata": false,
    "initielt": false,
    "dummy": false
}
fra miljø https://nvdbapiles-v3.atlas.vegvesen.no/


In [3]:
def hvilkeForeldreTyper( rel:dict): 
    returnval = '-99 Uten foreldre'
    
    if not pd.isnull( rel ) and 'foreldre' in rel: 
        temp = [ str( x['type']['id'] ) + ' ' + str( x['type']['navn'] )  for x in rel['foreldre'] ]
        temp.sort()
        returnval = ','.join( temp )
        
    return returnval 

def hvilkenForeldreId( rel:dict ): 
    returnval = ''

    if not pd.isnull( rel ) and 'foreldre' in rel: 
        if len( rel['foreldre'] ) == 1: 
            if len( rel['foreldre'][0]['vegobjekter']) == 1: 
                returnval = str( rel['foreldre'][0]['vegobjekter'][0] )
            else: 
                print(f"Fant {len(rel['foreldre'][0]['vegobjekter'])} mor-objekter" )
                tmp = [str(x) for x in rel['foreldre'][0]['vegobjekter'] ]
                returnval = ','.join( tmp )
        else: 
            print( f"Fant {rel['foreldre']} ulike typer mor-relasjon" )
                
    return returnval 

In [6]:
mydf['morObjektId'] = mydf['relasjoner'].apply( hvilkenForeldreId )

Fant 2 mor-objekter
Fant 2 mor-objekter
Fant 2 mor-objekter


In [7]:
mydf[ mydf['morObjektId'].str.contains( ',')]

,objekttype,nvdbId,versjon,startdato,Beskrivelse,relasjoner,veglenkesekvensid,detaljnivå,typeVeg,kommune,...,vref,veglenkeType,vegkategori,fase,vegnummer,relativPosisjon,adskilte_lop,trafikantgruppe,geometri,morObjektId
2245,794,198039680,3,2014-10-13,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",121785,Vegtrase og kjørebane,Enkel bilveg,4205,...,FV460 S2D1 m603,HOVED,F,V,460,0.091906,Nei,K,POINT Z(47407.06 6464344.067 4.089),"86263903,469959839"
2246,794,198039681,3,2014-10-13,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",121785,Vegtrase og kjørebane,Enkel bilveg,4205,...,FV460 S2D1 m602,HOVED,F,V,460,0.091793,Nei,K,POINT Z(47406.405 6464343.323 4.086),"86263906,469959840"
84160,794,514622591,1,2014-05-22,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221899, 'id': 201899...",404838,Vegtrase og kjørebane,Enkel bilveg,3020,...,KV24600 S1D1 m57,HOVED,K,V,24600,0.185685,Nei,K,POINT Z(264850.808 6637517.514 107.345),"83811218,716490600"


In [8]:
mydf['hvilkeForeldre'] = mydf['relasjoner'].apply( hvilkeForeldreTyper )

In [9]:
mydf['hvilkeForeldre'].value_counts( dropna=False )

95 Skiltpunkt                     44260
79 Stikkrenne/Kulvert             39624
83 Kum                            24948
87 Belysningspunkt                13316
98 Referansestolpe                 4882
7 Gjerde                           1349
20 Kantstolper/Refleks              870
-99 Uten foreldre                   801
174 Gangfelt                        742
25 Leskur                           632
3 Skjerm                            579
27 Renovasjon                       574
103 Fartsdemper                     419
23 Vegbom                           166
342 Trafikkspeil                    129
28 Utemøbler                        116
90 Signalpunkt                      111
519 Vegoppmerking, tverrgående      107
181 Lysmast                          99
502 Skap, teknisk                    82
465 Utgår_Matepunkt                  81
623 Stativ for turistinfo            63
22 Ferist                            62
163 Kamera                           23
24 Skiltportal                       22


# Trekker ut informasjon fra "Beskrivelse" - egenskap

Vi prøver to metoder: 
  * En der vi parser XML. Denne gir ganske mange feilsituasjoner fordi XML-dokumentet er blitt trunkert
  * En der vi bruker regulære uttrykk for å hente ut verdiene for taggene \<nord\>,\<oest\> og så videre
  
### Dekoder med regulære uttrykk

In [12]:
def dekodbeskrivelse( besk:str ): 
    
    try: 
        mydict = xmltodict.parse(besk )
    except ExpatError as e: 
        # print(f"Feilsituasjon {e} for input {besk} " )
        return { 'ExpatError' : str(e) }

    else: 
        norm = pd.json_normalize( mydict ).to_dict( orient='records')

        if isinstance( norm, list) and len( norm) == 1 and isinstance( norm[0], dict): 
            return norm[0]
        else:
            print( f"Klarte ikke dekode {besk} ")
    
    return { 'error' : 'empty'} 


def dekodbeskrivelse2( beskr:str, returtype='wkt' ):
    
    nord    = grabtekst( beskr, 'nord'  )
    oest    = grabtekst( beskr, 'oest' )
    hoyde   = grabtekst( beskr, 'hoyde' )
    mode    = grabtekst( beskr, 'mode' )    
    cq      = grabtekst( beskr, 'cq' ) 
    avstand = grabtekst( beskr, 'avstand' )
    
    if isinstance( nord, str) and isinstance( oest, str): 
        if isinstance( hoyde, str): 
            wkt = f"POINT Z( {oest} {nord} {hoyde} )" 
        else: 
            wkt = f"POINT Z( {oest} {nord} )" 
    elif isinstance( nord, list) and isinstance( oest, list):

        if isinstance( hoyde, list ): 
            koordstr = 'LINESTRING Z( '
            for ix, junk in enumerate( oest):
                if len( nord ) >= ix+1 and len( hoyde ) >= ix+1: 
                    koordstr += f" {oest[ix]} {nord[ix]} {hoyde[ix]}"
                    
                    if len( oest ) > ix+1: 
                        koordstr += ' , '
                        
        else: 
            koordstr = 'LINESTRING ( '
            for ix, junk in enumerate( oest):
                if len( nord ) >= ix+1 : 
                    koordstr += f" {oest[ix]} {nord[ix]}"
                    
                    if len( oest ) > ix+1: 
                        koordstr += ' , '
            
        if len( koordstr ) > len( 'LINESTRING Z(   ' ):
            koordstr += ' )'
            wkt = koordstr 
        else: 
            wkt = 'LINESTRING EMPTY'
                
    else: 
        wkt = 'POINT EMPTY'
        
    return { 'nord' : nord, 'oest' : oest, 'hoyde' : hoyde, 'mode' : mode, 'cq' : cq, 'avstand' : avstand, 'wkt' : wkt  }
    

def grabtekst( beskr:str, tagg:str ): 
    
    tmp = re.findall( f"<{tagg}>(.*?)</{tagg}>", beskr  )
    if len( tmp ) == 1: 
        return tmp[0]
    elif len( tmp ) > 1: 
        # print( f"Fant flere enn enn forekomst av {tagg} i {beskr}")
        return tmp
    else: 
        return None 
    
def godkjent( beskr:dict ): 
    
    if 'ExpatError' in beskr: 
        return 'ExpatError'
    elif 'vegreg_frapos.gnss_data.nord' in beskr and 'vegreg_tilpos.gnss_data.nord' in beskr: 
        return 'Linje'
    elif 'vegreg_frapos.gnss_data.nord' in beskr: 
        return 'FRApos'
    elif 'vegreg_tilpos.gnss_data.nord' in beskr:         
        return 'TILpos'
    elif 'vegreg_frapos' in beskr and beskr['vegreg_frapos'] == None: 
        return 'No data'
    else: 
        return 'Godkjent'

In [13]:
mydf['Beskrivelse'].fillna( '<vegreg_frapos/>', inplace=True)

mydf['dekodet'] = mydf['Beskrivelse'].apply( dekodbeskrivelse2 )

mydf['dekodet_geom'] = mydf['dekodet'].apply( lambda x : x['wkt'] )

### Dekoder med XML-parsing 

In [14]:
mydf['Beskrivelse'].fillna( '<vegreg_frapos/>', inplace=True)

mydf['Beskrivelse_normalisert'] = mydf['Beskrivelse'].apply(  dekodbeskrivelse )

mydf['godkjent'] = mydf['Beskrivelse_normalisert'].apply( godkjent ) 

mydf['godkjent'].value_counts()

FRApos        120541
ExpatError     12332
TILpos          1167
No data           79
Name: godkjent, dtype: int64

In [15]:
mydf['geomtype'] = mydf['dekodet_geom'].apply( lambda x: x.split( '(' )[0] )

In [16]:
mydf['geomtype'].value_counts()

POINT Z        125558
POINT EMPTY      7320
LINESTRING       1241
Name: geomtype, dtype: int64

In [17]:
b = mydf.groupby( ['geomtype', 'hvilkeForeldre']).agg( {'nvdbId' : 'count'}).reset_index()

In [18]:
b[ b['geomtype'].str.contains( 'LINESTRING')]

,geomtype,hvilkeForeldre,nvdbId
0,LINESTRING,-99 Uten foreldre,2
1,LINESTRING,20 Kantstolper/Refleks,382
2,LINESTRING,3 Skjerm,273
3,LINESTRING,7 Gjerde,427
4,LINESTRING,95 Skiltpunkt,157


# TODO - Kvalitetssikring og filtrering

### Tar kun med objekter der vi har hatt suksess med å lage punkt-geometri 

Og der avstanden mellom NVDB senterlinjegeometr - den nye punktgeometrien < 10 meter

Og der metadata om avstand samsvarer noenlunde greit (< 5m differanse) med den avstanden vi finner mellom geometriene

### Blir 119265 objekter

In [121]:
godkjente = mydf[ mydf['dekodet_geom'].str.contains( 'POINT')].copy()

godkjente = godkjente[ ~godkjente['morObjektId'].str.contains(',')] # Ignorerer dem med flere enn ett morobjekt

godkjente = godkjente[ godkjente['morObjektId'] != ''] # Ignorerer dem uten morobjekt
godkjente = godkjente[ ~godkjente['morObjektId'].isnull()].copy() 

godkjente['morObjektId'] = godkjente['morObjektId'].astype(int)

godkjente['geometry_nvdb'] = mydf['geometri'].apply( wkt.loads )

godkjente['geometry_dekodet'] = mydf['dekodet_geom'].apply( wkt.loads )

In [93]:
def sjekkavstand( row, geomcolA='geometry_nvdb', geomcolB='geometry_dekodet' ): 
    """
    Sjekker avstand mellom to kolonner
    """
    
    return row[geomcolA].distance(row[geomcolB])

In [122]:
godkjente['avstand_dekodet'] = godkjente.apply( sjekkavstand, axis=1  )

godkjente['avstand_metadata'] = godkjente['dekodet'].apply( lambda x : float( x['avstand'] if x['avstand'] else np.nan ))

godkjente['avstand_diff'] = abs( godkjente['avstand_metadata'] - godkjente['avstand_dekodet'] )

godkjente = godkjente[ (godkjente['avstand_dekodet'] < 10) & (godkjente['avstand_diff'] < 5) ]

In [124]:
godkjente['morObjektId']

0           91782211
1           91782207
4           80806802
5           91868143
6           91868140
             ...    
134114    1013685510
134115    1013817159
134116    1013847171
134117    1013945854
134118    1014251820
Name: morObjektId, Length: 119265, dtype: int64

# TODO: Forbered endringssett-informasjon

### Itererer og henter de objekttypene som trengs

### Henter datakatalog-definisjoner og mor-objekt detaljer

In [95]:
foreldreliste = list( mydf['hvilkeForeldre'].unique() )
# Lager datakatalog-struktur for geometriegenskaper for de objekttypene vi har

dakat = { }
for forelder in foreldreliste: 
    objTypeId = forelder.split()[0]
    dakat[objTypeId] = {}
    if objTypeId != '-99': 
        tmp = requests.get( 'https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekttyper/' + objTypeId + '.json' ).json()
        geometriegenskaper = [ x for x in tmp['egenskapstyper'] if 'eometri' in x['navn']]
        if len( geometriegenskaper) != 1: 
            print( f"fant {len(geometriegenskaper)}  geometrityper {forelder} ", 
                  [ x['navn'] for x in geometriegenskaper  ])
        
        for geom in geometriegenskaper: 
        
           dakat[objTypeId][geom['navn']] = geom
            

    else: 
        pass 
    # Trenger ingen dummy datastrktur for -99 ingenforeldre
        
#         dakat[-99] = { 'id' : -99, 'navn' : 'Foreldreløs og irrelevant'  }
        # Legg inn enerannen dummy struktur for -99 ingenforeldre

fant 3  geometrityper 174 Gangfelt  ['Geometri, punkt', 'Geometri, linje', 'Geometri, flate']
fant 3  geometrityper 79 Stikkrenne/Kulvert  ['Geometri, punkt', 'Geometri, linje', 'Geometri, flate']
fant 3  geometrityper 25 Leskur  ['Geometri, punkt', 'Geometri, flate', 'Utgår_Geometri, linje']
fant 2  geometrityper 23 Vegbom  ['Geometri, punkt', 'Geometri, linje']
fant 2  geometrityper 22 Ferist  ['Geometri, punkt', 'Geometri, linje']
fant 2  geometrityper 519 Vegoppmerking, tverrgående  ['Geometri, punkt', 'Geometri, linje']
fant 2  geometrityper 24 Skiltportal  ['Geometri, punkt', 'Geometri, linje']
fant 3  geometrityper 65 Bygning  ['Geometri, punkt', 'Utgår_Geometri, linje', 'Geometri, flate']
fant 3  geometrityper 73 Sikringsbolt  ['Geometri, punkt', 'Geometri, linje', 'Geometri, flate']


In [104]:
dfListe = []

for objTypeNavn in list( godkjente['hvilkeForeldre'].unique() ): 
    objTypeId =  objTypeNavn.split()[0]
    if objTypeId != '-99': 
        
        geomfilter = ''
        # Lager filter for kun å ta med de objektene som IKKE har noen form for egengeometri
        count = 0
        for geomtype in dakat[objTypeId].keys():
            if count >= 1: 
                geomfilter += ' AND '
            geomfilter += f"({dakat[objTypeId][geomtype]['id']}=null)"

            count += 1 
            #   Eksempler: egenskap=(4723=null) AND (8911=null)        
            #              egenskap=(4794=null)

            
        geomfilter += ' AND (relasjon(794))'
        print( f"Filter for {objTypeNavn}: {geomfilter}")
        sok = nvdbapiv3.nvdbFagdata( int( objTypeId ))
        sok.filter( { 'egenskap' : geomfilter } )

        tempDf = pd.DataFrame( sok.to_records( vegsegmenter=False ))
        col = ['objekttype', 'nvdbId', 'versjon', 'startdato', 'relasjoner', 'vegsystemreferanser', 'geometri']
        if len( tempDf ) > 0: 
            dfListe.append( tempDf[col].copy() )
        else: 
            print( f"Fant ingen objekter av typen {objTypeNavn} med filter {geomfilter}" )

        

Filter for 95 Skiltpunkt: (4794=null) AND (relasjon(794))
Eksport av 33371 objekter kommer til å ta tid...
Objekt 1000 av 33371
Objekt 5000 av 33371
Objekt 10000 av 33371
Objekt 20000 av 33371
Objekt 30000 av 33371
Objekt 40000 av 33371
Objekt 50000 av 33371
Filter for 174 Gangfelt: (4836=null) AND (9434=null) AND (12089=null) AND (relasjon(794))
Filter for 83 Kum: (4784=null) AND (relasjon(794))
Eksport av 14760 objekter kommer til å ta tid...
Objekt 1000 av 14760
Objekt 5000 av 14760
Objekt 10000 av 14760
Filter for 7 Gjerde: (4716=null) AND (relasjon(794))
Objekt 1000 av 1454
Filter for 3 Skjerm: (4712=null) AND (relasjon(794))
Filter for 103 Fartsdemper: (4801=null) AND (relasjon(794))
Filter for 27 Renovasjon: (4735=null) AND (relasjon(794))
Filter for 25 Leskur: (4733=null) AND (9844=null) AND (9427=null) AND (relasjon(794))
Filter for 98 Referansestolpe: (4797=null) AND (relasjon(794))
Objekt 1000 av 2100
Filter for 79 Stikkrenne/Kulvert: (4780=null) AND (5899=null) AND (5902=nu

In [112]:
vegobjekter = pd.concat( dfListe, axis=0, ignore_index=True )

# Fjerner relasjonsløse objekter, vi får en del falske treff på det fra NVDB api LES
vegobjekter = vegobjekter[ ~vegobjekter.relasjoner.isnull()].copy()

In [115]:
vegobjekter = vegobjekter.add_prefix( 'vegobj_' )

In [126]:
med_datter = godkjente.merge( vegobjekter, how='inner', left_on='morObjektId', right_on='vegobj_nvdbId' )

In [128]:
med_datter['vegobj_geometry'] = med_datter['vegobj_geometri'].apply( wkt.loads )

In [136]:
med_datter['avst_diff_vegobj'] = med_datter.apply( lambda x : sjekkavstand( x, geomcolA='geometry_nvdb', geomcolB='vegobj_geometry' ), axis=1)

In [148]:
med_datter['avst_diff_vegobj_egengeom'] = med_datter.apply( lambda x : sjekkavstand( x, geomcolA='geometry_dekodet', geomcolB='vegobj_geometry' ), axis=1)

In [137]:
med_datter[ med_datter['avst_diff_vegobj'] > 2]

,objekttype,nvdbId,versjon,startdato,Beskrivelse,relasjoner,veglenkesekvensid,detaljnivå,typeVeg,kommune,...,avstand_diff,vegobj_objekttype,vegobj_nvdbId,vegobj_versjon,vegobj_startdato,vegobj_relasjoner,vegobj_vegsystemreferanser,vegobj_geometri,vegobj_geometry,avst_diff_vegobj
123,794,169544536,2,2009-07-15,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",886036,Vegtrase og kjørebane,Enkel bilveg,1870,...,0.069438,95,89242830,4,2021-07-15,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",FV822 S2D1 m1601,POINT Z(516528.103360753 7610815.99564179 69.831),POINT Z (516528.103360753 7610815.99564179 69....,2.520889
124,794,169544537,2,2009-07-15,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",886036,Vegtrase og kjørebane,Enkel bilveg,1870,...,0.069438,95,89242826,4,2021-07-15,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",FV822 S2D1 m1601,POINT Z(516523.491668548 7610817.95963184 69.899),POINT Z (516523.491668548 7610817.95963184 69....,2.621570
125,794,169544539,2,2009-07-15,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",886036,Vegtrase og kjørebane,Enkel bilveg,1870,...,0.628104,95,89242770,4,2021-07-15,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",FV822 S2D1 m3610,POINT Z(515340.579545935 7609410.17112655 49.102),POINT Z (515340.579545935 7609410.17112655 49....,3.036216
126,794,169544540,2,2009-07-15,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",886036,Vegtrase og kjørebane,Enkel bilveg,1870,...,0.619476,95,89242765,4,2021-07-15,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",FV822 S2D1 m3610,POINT Z(515338.458041673 7609416.77964751 49.086),POINT Z (515338.458041673 7609416.77964751 49....,3.918710
127,794,169544542,2,2009-07-16,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",886037,Vegtrase og kjørebane,Enkel bilveg,1866,...,1.239080,95,89242575,5,2021-07-15,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",FV822 S3D1 m2403,POINT Z(512150.167388752 7606673.20637297 68.201),POINT Z (512150.167388752 7606673.20637297 68....,3.132502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76579,794,844113625,1,2017-11-24,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",2793343,Vegtrase og kjørebane,Rundkjøring,3001,...,0.108388,95,844113621,2,2022-02-16,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",RV204 S1D1 m0 KD1 m165,POINT (286660.91731953504 6560255.5696113305),POINT (286660.91731953504 6560255.5696113305),11.377336
76580,794,844113630,1,2017-11-24,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",2793343,Vegtrase og kjørebane,Rundkjøring,3001,...,0.327732,95,844113545,2,2022-02-16,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",RV204 S1D1 m0 KD1 m21,POINT (286689.67453850876 6560268.812461292),POINT (286689.67453850876 6560268.812461292),6.267148
76581,794,844113631,1,2017-11-24,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",2793343,Vegtrase og kjørebane,Rundkjøring,3001,...,0.030439,95,844113548,2,2022-02-16,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",RV204 S1D1 m0 KD1 m37,POINT (286702.95713590423 6560279.634197602),POINT (286702.95713590423 6560279.634197602),12.573253
76583,794,844113648,1,2017-11-24,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",971783,Vegtrase,Kanalisert veg,3001,...,0.262510,95,844113646,2,2022-02-16,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",RV204 S1D1 m183,POINT (286832.91416299477 6560160.131930016),POINT (286832.91416299477 6560160.131930016),10.585306


In [140]:
med_datter.iloc[0]['vegobj_relasjoner']

{'barn': [{'listeid': 220004,
   'id': 200004,
   'type': {'id': 96, 'navn': 'Skiltplate'},
   'vegobjekter': [91782208, 91782210]},
  {'listeid': 221787,
   'id': 201787,
   'type': {'id': 794, 'navn': 'Systemobjekt'},
   'vegobjekter': [166855154]}]}

In [183]:
def tellRelasjoner( rel:dict ): 
    
    countForeldre = 0
    countBarn = 0
    if 'barn' in rel: 
#         for barn in rel['barn']:
#             countBarn += len( barn['vegobjekter'])
        countBarn = len( rel['barn'])
        
    if 'foreldre' in rel: 
        for forelder in rel['foreldre']: 
            countForeldre += len( forelder['vegobjekter'])            


    return countBarn 

In [173]:
med_datter['vegobj_hvilkeforeldre'] = med_datter['vegobj_relasjoner'].apply( hvilkeForeldreTyper  )

In [184]:
med_datter['antallBarn'] = med_datter['vegobj_relasjoner'].apply( tellRelasjoner )

In [181]:
med_datter.groupby( ['hvilkeForeldre', 'vegobj_hvilkeforeldre']).agg( { 'vegobj_nvdbId' : 'count'})

vegobj_nvdbId
hvilkeForeldre                 vegobj_hvilkeforeldre                      
103 Fartsdemper                -99 Uten foreldre                       379
                               174 Gangfelt                              5
163 Kamera                     -99 Uten foreldre                        18
                               67 Tunnelløp                              2
165 Vindmåler                  -99 Uten foreldre                         1
174 Gangfelt                   -99 Uten foreldre                       638
181 Lysmast                    -99 Uten foreldre                        79
                               87 Belysningspunkt                       17
20 Kantstolper/Refleks         -99 Uten foreldre                       318
                               67 Tunnelløp                              9
210 Pumpestasjon               -99 Uten foreldre                         3
22 Ferist                      -99 Uten foreldre                        33
23 Vegbom                      -99 Uten foreldre                       129
                               107 Værutsatt veg                         8
                               67 Tunnelløp                              5
24 Skiltportal                 -99 Uten foreldre                        14
25 Leskur                      -99 Uten foreldre                         3
                               487 Holdeplassutrustning                548
27 Renovasjon                  -99 Uten foreldre                       237
                               39 Rasteplass                            13
                               487 Holdeplassutrustning                269
28 Utemøbler                   -99 Uten foreldre                        39
                               39 Rasteplass                            45
                               487 Holdeplassutrustning                 20
3 Skjerm                       -99 Uten foreldre                       121
342 Trafikkspeil               -99 Uten foreldre                       115
465 Utgår_Matepunkt            -99 Uten foreldre                         2
502 Skap, teknisk              -99 Uten foreldre                         1
519 Vegoppmerking, tverrgående -99 Uten foreldre                        64
542 Støtpute                   -99 Uten foreldre                         3
                               5 Rekkverk                                6
623 Stativ for turistinfo      -99 Uten foreldre                        54
                               39 Rasteplass                             7
65 Bygning                     -99 Uten foreldre                        11
                               39 Rasteplass                             2
                               581 Tunnel                                1
7 Gjerde                       -99 Uten foreldre                       801
73 Sikringsbolt                72 Bergsikring                            6
79 Stikkrenne/Kulvert          -99 Uten foreldre                     18851
83 Kum                         -99 Uten foreldre                     13973
                               67 Tunnelløp                             18
                               78 Lukket rørgrøft                       30
                               79 Stikkrenne/Kulvert                    61
85 Pumpe                       -99 Uten foreldre                         4
87 Belysningspunkt             -99 Uten foreldre                      4047
                               86 Belysningsstrekning                 1397
90 Signalpunkt                 -99 Uten foreldre                        11
                               67 Tunnelløp                              2
                               67 Tunnelløp,89 Signalanlegg             12
                               89 Signalanlegg                          68
95 Skiltpunkt                  -99 Uten foreldre                     32092
                               24 Skiltportal                            3
           

In [182]:
med_datter['hvilkeForeldre'].value_counts()

95 Skiltpunkt                     32161
79 Stikkrenne/Kulvert             18851
83 Kum                            14082
87 Belysningspunkt                 5444
98 Referansestolpe                 1952
7 Gjerde                            801
174 Gangfelt                        638
25 Leskur                           551
27 Renovasjon                       519
103 Fartsdemper                     384
20 Kantstolper/Refleks              327
23 Vegbom                           142
3 Skjerm                            121
342 Trafikkspeil                    115
28 Utemøbler                        104
181 Lysmast                          96
90 Signalpunkt                       93
519 Vegoppmerking, tverrgående       64
623 Stativ for turistinfo            61
22 Ferist                            33
163 Kamera                           20
24 Skiltportal                       14
65 Bygning                           14
542 Støtpute                          9
73 Sikringsbolt                       6


In [185]:
med_datter['antallBarn'].describe()

count    76614.000000
mean         1.960099
std          0.786032
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          5.000000
Name: antallBarn, dtype: float64

In [172]:
tmp[ tmp > 0].describe()

count    2621.000000
mean        1.008012
std         0.089169
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: antallRelasjoner, dtype: float64

In [150]:
med_datter['avst_diff_vegobj_egengeom'].describe()

count    76614.000000
mean         1.770781
std          2.214005
min          0.000000
25%          0.874192
50%          1.340981
75%          2.066270
max        323.748904
Name: avst_diff_vegobj_egengeom, dtype: float64

In [152]:
med_datter['avst_diff_vegobj_egengeom'].quantile( q=[0.75, .95, .99, .999, .9999])

0.7500     2.066270
0.9500     4.617020
0.9900     8.081992
0.9990    17.273243
0.9999    68.296346
Name: avst_diff_vegobj_egengeom, dtype: float64

In [205]:
r = med_datter['kommune'].value_counts()
r[ r < 100] # Prøver kommune 1848 

1577    97
5444    95
1865    93
3447    92
1848    92
        ..
4227     1
3043     1
5014     1
4628     1
1566     1
Name: kommune, Length: 189, dtype: int64

In [199]:
myset = set([1, 2, 3] )

In [200]:
list_of_lists = [ [1, 2, 3, 4], [4, 5], [1, 2, 3, 4, 5, 6] ]

In [201]:
for elem in list_of_lists: 
    myset.update(elem)
myset

{1, 2, 3, 4, 5, 6}

# Eksempel på at både mor og datter finnes i datasettet

Dette Skiltpunktet [540008156](https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekt.json?id=540008156&dybde=full) og dette variable skiltet [540008149](https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/97/540008149/1) inngår i samme relasjonstre, og begge har hvert sitt systemobjekt - med identiske "Beskrivelse"-egenskaper. 

[Vegkart-søk](https://vegkart.atlas.vegvesen.no/#valgt:540008156:95)

In [216]:
med_datter[ med_datter['vref'].str.contains('FV156 S1D1')] # FV156 S1D1 m1160

,objekttype,nvdbId,versjon,startdato,Beskrivelse,relasjoner,veglenkesekvensid,detaljnivå,typeVeg,kommune,...,vegobj_startdato,vegobj_relasjoner,vegobj_vegsystemreferanser,vegobj_geometri,vegobj_geometry,avst_diff_vegobj,avst_diff_vegobj_egengeom,antallRelasjoner,vegobj_hvilkeforeldre,antallBarn
1655,794,197780920,2,2014-07-16,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",444273,Vegtrase og kjørebane,Enkel bilveg,3021,...,2016-03-29,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",FV156 S1D1 m8392,POINT Z(260380.6 6629127.056 3.29),POINT Z (260380.6 6629127.056 3.29),0.000000,1.438042,"(0, 3)",-99 Uten foreldre,2
1656,794,197780921,1,2008-10-20,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",444273,Vegtrase og kjørebane,Enkel bilveg,3021,...,2016-03-29,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",FV156 S1D1 m8391,POINT Z(260380.901 6629127.267 3.289),POINT Z (260380.901 6629127.267 3.289),0.000000,2.206528,"(0, 3)",-99 Uten foreldre,2
1657,794,197780922,1,2008-10-20,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",444273,Vegtrase og kjørebane,Enkel bilveg,3021,...,2016-03-29,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",FV156 S1D1 m8915,POINT Z(260091.759 6628715.679 7.033),POINT Z (260091.759 6628715.679 7.033),0.000000,6.329727,"(0, 3)",-99 Uten foreldre,2
1658,794,197780923,1,2008-10-20,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",444273,Vegtrase og kjørebane,Enkel bilveg,3021,...,2016-03-29,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",FV156 S1D1 m8916,POINT Z(260091.603 6628714.742 7.035),POINT Z (260091.603 6628714.742 7.035),0.000000,6.340344,"(0, 3)",-99 Uten foreldre,2
13442,794,364667703,1,2012-11-06,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221792, 'id': 201792...",2101020,Vegtrase og kjørebane,Gang- og sykkelveg,3020,...,2012-11-06,"{'barn': [{'listeid': 221792, 'id': 201792, 't...",FV156 S1D125 m45,POINT Z(264810.685 6632295.299 69.894),POINT Z (264810.685 6632295.299 69.894),0.000000,0.865813,"(0, 1)",-99 Uten foreldre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44822,794,540008439,1,2014-08-14,<vegreg_frapos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221787, 'id': 201787...",444228,Vegtrase og kjørebane,Rundkjøring,3021,...,2022-02-28,"{'barn': [{'listeid': 220004, 'id': 200004, 't...",EV18 S57D1 m8512 KD4 m230,POINT (263151.3090518024 6631843.488514901),POINT (263151.3090518024 6631843.488514901),8.448487,9.294514,"(0, 2)",-99 Uten foreldre,2
44824,794,540978878,1,2014-08-19,<vegreg_tilpos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221788, 'id': 201788...",2437260,Vegtrase og kjørebane,Gang- og sykkelveg,3021,...,2014-08-19,"{'barn': [{'listeid': 221788, 'id': 201788, 't...",FV156 S1D128 m66-82,"LINESTRING Z(262646.508 6630022.329 58.256, 26...","LINESTRING Z (262646.508 6630022.329 58.256, 2...",0.000000,1.372391,"(0, 1)",-99 Uten foreldre,1
44825,794,540978881,1,2014-08-19,<vegreg_tilpos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221788, 'id': 201788...",2437260,Vegtrase og kjørebane,Gang- og sykkelveg,3021,...,2014-08-19,"{'barn': [{'listeid': 221788, 'id': 201788, 't...",FV156 S1D128 m66-82,"LINESTRING Z(262646.508 6630022.329 58.256, 26...","LINESTRING Z (262646.508 6630022.329 58.256, 2...",0.000000,0.996398,"(0, 1)",-99 Uten foreldre,1
44826,794,540978912,1,2014-08-19,<vegreg_tilpos><gnss_data><talker>GN</talker><...,"{'foreldre': [{'listeid': 221788, 'id': 201788...",2437260,Vegtrase og kjørebane,Gang- og sykkelveg,3021,...,2014-08-19,"{'barn': [{'listeid': 221788, 'id': 201788, 't...",FV156 S1D128 m143-184,"LINESTRING Z(262661.13 6629948.576 62.139, 262...","LINESTRING Z (262661.13 6629948.576 62.139, 26...",0.000000,0.558168,"(0,

In [207]:
med_datter.columns

Index(['objekttype', 'nvdbId', 'versjon', 'startdato', 'Beskrivelse',
       'relasjoner', 'veglenkesekvensid', 'detaljnivå', 'typeVeg', 'kommune',
       'fylke', 'vref', 'veglenkeType', 'vegkategori', 'fase', 'vegnummer',
       'relativPosisjon', 'adskilte_lop', 'trafikantgruppe', 'geometri',
       'morObjektId', 'hvilkeForeldre', 'dekodet', 'dekodet_geom',
       'Beskrivelse_normalisert', 'godkjent', 'geomtype', 'geometry_nvdb',
       'geometry_dekodet', 'avstand_dekodet', 'avstand_metadata',
       'avstand_diff', 'vegobj_objekttype', 'vegobj_nvdbId', 'vegobj_versjon',
       'vegobj_startdato', 'vegobj_relasjoner', 'vegobj_vegsystemreferanser',
       'vegobj_geometri', 'vegobj_geometry', 'avst_diff_vegobj',
       'avst_diff_vegobj_egengeom', 'antallRelasjoner',
       'vegobj_hvilkeforeldre', 'antallBarn'],
      dtype='object')

In [217]:
# Kreativ feilsøking
mittObj = requests.get( 'https://nvdbapiles-v3.atlas.vegvesen.no/vegobjekter/97/540008149/1.json').json()


### Finnes belysningstrekning i relasjonstreet?

In [226]:
med_datter[ med_datter['hvilkeForeldre'].str.contains('86')]

,objekttype,nvdbId,versjon,startdato,Beskrivelse,relasjoner,veglenkesekvensid,detaljnivå,typeVeg,kommune,...,vegobj_startdato,vegobj_relasjoner,vegobj_vegsystemreferanser,vegobj_geometri,vegobj_geometry,avst_diff_vegobj,avst_diff_vegobj_egengeom,antallRelasjoner,vegobj_hvilkeforeldre,antallBarn


In [223]:
list( tmp.keys())[0]

95

# Finnes egengeometri, punkt for alle dataene våre? 

In [227]:
forb = nvdbapiv3.apiforbindelse()
dakat1 = forb.les( '/vegobjekttyper', 
                      params={'inkluder' : 'egenskapstyper'} ).json()

dakat = { }
for objType in dakat1: 
    dakat[objType['id']] = {}
    geometriegenskaper = [ x for x in objType['egenskapstyper'] if 'eometri' in x['navn']]
    for geom in geometriegenskaper: 
        dakat[objType['id']][geom['navn']] = geom['id']
            

In [239]:
kandidater = list( mydf[ ~mydf['hvilkeForeldre'].str.contains('-99') ]['hvilkeForeldre'].unique())
geomtyper = []

for kandidat in kandidater:
    objTypeId = int( kandidat.split()[0] )

    geomtyper.append( kandidat + ': ' +  ' og '.join(  dakat[objTypeId].keys() ) )  
        
geomtyper

['95 Skiltpunkt: Geometri, punkt',
 '174 Gangfelt: Geometri, punkt og Geometri, linje og Geometri, flate',
 '83 Kum: Geometri, punkt',
 '7 Gjerde: Geometri, linje',
 '3 Skjerm: Geometri, linje',
 '79 Stikkrenne/Kulvert: Geometri, punkt og Geometri, linje og Geometri, flate',
 '27 Renovasjon: Geometri, punkt',
 '103 Fartsdemper: Geometri, punkt',
 '25 Leskur: Geometri, punkt og Geometri, flate og Utgår_Geometri, linje',
 '98 Referansestolpe: Geometri, punkt',
 '90 Signalpunkt: Geometri, punkt',
 '23 Vegbom: Geometri, punkt og Geometri, linje',
 '87 Belysningspunkt: Geometri, punkt',
 '85 Pumpe: Geometri, punkt',
 '22 Ferist: Geometri, punkt og Geometri, linje',
 '20 Kantstolper/Refleks: Geometri, linje',
 '519 Vegoppmerking, tverrgående: Geometri, punkt og Geometri, linje',
 '24 Skiltportal: Geometri, punkt og Geometri, linje',
 '342 Trafikkspeil: Geometri, punkt',
 '28 Utemøbler: Geometri, punkt',
 '465 Utgår_Matepunkt: Utgår_Geometri, punkt',
 '163 Kamera: Geometri, punkt',
 '623 Stat

In [243]:
geomtyper2 = { int( x.split()[0]) : x for x in geomtyper  }

In [244]:
geomtyper2

{95: '95 Skiltpunkt: Geometri, punkt',
 174: '174 Gangfelt: Geometri, punkt og Geometri, linje og Geometri, flate',
 83: '83 Kum: Geometri, punkt',
 7: '7 Gjerde: Geometri, linje',
 3: '3 Skjerm: Geometri, linje',
 79: '79 Stikkrenne/Kulvert: Geometri, punkt og Geometri, linje og Geometri, flate',
 27: '27 Renovasjon: Geometri, punkt',
 103: '103 Fartsdemper: Geometri, punkt',
 25: '25 Leskur: Geometri, punkt og Geometri, flate og Utgår_Geometri, linje',
 98: '98 Referansestolpe: Geometri, punkt',
 90: '90 Signalpunkt: Geometri, punkt',
 23: '23 Vegbom: Geometri, punkt og Geometri, linje',
 87: '87 Belysningspunkt: Geometri, punkt',
 85: '85 Pumpe: Geometri, punkt',
 22: '22 Ferist: Geometri, punkt og Geometri, linje',
 20: '20 Kantstolper/Refleks: Geometri, linje',
 519: '519 Vegoppmerking, tverrgående: Geometri, punkt og Geometri, linje',
 24: '24 Skiltportal: Geometri, punkt og Geometri, linje',
 342: '342 Trafikkspeil: Geometri, punkt',
 28: '28 Utemøbler: Geometri, punkt',
 465: '

In [242]:
mydf['morObjektTypeId'] = mydf['hvilkeForeldre'].apply( lambda x : int( x.split()[0]))

In [245]:
mydf['morObjektBeskrivelse'] = mydf['morObjektTypeId'].map( geomtyper2).fillna('Ubrukelige')
# df['Country'] = df['Birth City'].map(city_dict).fillna('Other')

In [247]:
brukdisse = [27, 28, 83, 85, 87, 90, 95, 97, 98, 163, 165, 181, 210, 342, 461, 481, 502, 542, 623 ]

In [250]:
mydf[ mydf['morObjektTypeId'].isin( brukdisse) ]['morObjektBeskrivelse'].value_counts()

95 Skiltpunkt: Geometri, punkt                44260
83 Kum: Geometri, punkt                       24948
87 Belysningspunkt: Geometri, punkt           13316
98 Referansestolpe: Geometri, punkt            4882
27 Renovasjon: Geometri, punkt                  574
342 Trafikkspeil: Geometri, punkt               129
28 Utemøbler: Geometri, punkt                   116
90 Signalpunkt: Geometri, punkt                 111
181 Lysmast: Geometri, punkt                     99
502 Skap, teknisk: Geometri, punkt               82
623 Stativ for turistinfo: Geometri, punkt       63
163 Kamera: Geometri, punkt                      23
542 Støtpute: Geometri, punkt                    14
165 Vindmåler: Geometri, punkt                    5
210 Pumpestasjon: Geometri, punkt                 4
481 Tennpunkt: Geometri, punkt                    4
85 Pumpe: Geometri, punkt                         4
461 Elektrisk anlegg: Geometri, punkt             2
97 Variabelt skilt: Geometri, punkt               1
Name: morObj